# Chapter 6 (continuation) - Analysis of Variance (ANOVA)

In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from math import sqrt

### ANOVA (Analysis of Variance)

The **Analysis of Variance (ANOVA)** is a statistical test used to determine if there are significant differences between the means of three or more independent groups.

---

#### Key Concepts:
1. **Purpose**:  
   ANOVA tests whether the group means are significantly different from each other.

2. **When to Use**:  
   - You have **three or more independent groups**.
   - The dependent variable is measured on a **continuous scale**.
   - The independent variable is **categorical** with multiple levels.

3. **Hypotheses**:
   - **Null Hypothesis $( H_0 )$**:  
     All group means are equal:  
     $
     \mu_1 = \mu_2 = \mu_3 = \dots = \mu_k
     $
   - **Alternative Hypothesis $( H_1 )$**:  
     At least one group mean is significantly different.

4. **Assumptions**:
   - The data are approximately **normally distributed**.
   - The variances of the groups are **homogeneous** (equal variance).
   - The observations are **independent**.

---

#### How ANOVA Works:
1. Calculate the **total variability** in the data (Total Sum of Squares, \( SS_{total} \)).
2. Partition the total variability into:
   - **Between-Group Variability** $( SS_{between} $): Variability due to differences between group means.
   - **Within-Group Variability** $( SS_{within} $): Variability due to differences within each group.

3. Compute the **F-statistic**:
   $
   F = \frac{\text{Mean Square Between Groups}}{\text{Mean Square Within Groups}}
   $
   Where:
   - $ MS_{between} = SS_{between} / (k - 1) $
   - $ MS_{within} = SS_{within} / (N - k) $
   - $ k $: Number of groups
   - $ N $: Total number of observations

4. Compare the $ F $-statistic to a critical value from the $ F $-distribution (or use a p-value).

---

#### Output and Interpretation:
1. **p-value**:
   - If $ p < \alpha $ (e.g., $ \alpha = 0.05 $), reject $ H_0 $.
   - This indicates that at least one group mean is significantly different.
   
2. **Post-hoc Tests**:
   - If $ H_0 $ is rejected, perform post-hoc tests (e.g., Tukey’s HSD) to determine which group(s) differ.

---

#### Example Use Case:
You want to compare the test scores of students across three different teaching methods (Group A, Group B, Group C). ANOVA helps determine if the teaching method has a significant effect on the scores.

| Model 5 | Model Centurion | Model 8 |
|:-------:|:---------------:|:-------:|
|    14   |       17        |    24   |
|    17   |       19        |    33   |
|    12   |       19        |    28   |
|    14   |       22        |    22   |
|    22   |       18        |    29   |
|    19   |       17        |    32   |
|    16   |       18        |    31   |
|    17   |       19        |    28   |
|    15   |       17        |    29   |
|    16   |       16        |    30   |


In [5]:
# Define arrays for the three models
five = np.array([14, 17, 12, 14, 22, 19, 16, 17, 15, 16])       # Data for Model 5
centurion = np.array([17, 19, 19, 22, 18, 17, 18, 19, 17, 16])  # Data for Model Centurion
eight = np.array([24, 33, 28, 22, 29, 32, 31, 28, 29, 30])      # Data for Model 8

# Calculate the mean for each model
mean_five = five.mean()                                         # Mean of Model 5
mean_centurion = centurion.mean()                               # Mean of Model Centurion
mean_eight = eight.mean()                                       # Mean of Model 8

# Print the means with descriptive labels
print(f"        Model 5 Mean: {mean_five:.2f}")                 # Print mean for Model 5
print(f"Model Centurion Mean: {mean_centurion:.2f}")            # Print mean for Model Centurion
print(f"        Model 8 Mean: {mean_eight:.2f}")                # Print mean for Model 8

        Model 5 Mean: 16.20
Model Centurion Mean: 18.20
        Model 8 Mean: 28.60


In [6]:
from scipy.stats import f_oneway

# Perform one-way ANOVA on the three models
fstat, p = f_oneway(five, centurion, eight)

# Display the F-statistic and p-value
print('F-statistic = {0:.4f}, p-value = {1:.4f}'.format(fstat, p))  

F-statistic = 59.3571, p-value = 0.0000


    Since p (0.0000) < α (0.05), we reject the null hypothesis (H0: mu1 = mu2 = m3). This result suggests that the mean performance of at least one of the models (Model 5, Model Centurion, or Model 8) is significantly different from the others. 

In [8]:
toasters = pd.DataFrame({'five': five,
                        'centurion': centurion,
                        'eight': eight})

toasters_melt = pd.melt(toasters.reset_index(),
                        id_vars=['index'],
                        value_vars=toasters.columns,
                        var_name='toastermodel',
                        value_name='excesshours')

toasters_melt.head(12)

,index,toastermodel,excesshours
0,0,five,14
1,1,five,17
2,2,five,12
3,3,five,14
4,4,five,22
5,5,five,19
6,6,five,16
7,7,five,17
8,8,five,15
9,9,five,16


We will a model that explains the excess hours as a function of the toaster model.

In [10]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model1 = ols('excesshours ~ C(toastermodel)', data=toasters_melt).fit()  # C makes clear to Python that toastmodel is categorical

anova_toasters = sm.stats.anova_lm(model1, typ=2)

anova_toasters

,sum_sq,df,F,PR(>F)
C(toastermodel),886.4,2.0,59.357143,1.306568e-10
Residual,201.6,27.0,NaN,NaN


F-statistic: 𝐹 = 59.3571

    This is a very high F-statistic, suggesting that the between-group variability is much greater than the within-group variability.

p-value: 1.31×10^−10
 
    The p-value is much smaller than 𝛼 = 0.05. Therefore, we reject the null hypothesis and conclude that at least one toaster model's mean performance is significantly different.

In [12]:
# A function to calculate the sum of squares and the totals

def anova_summary(aov):
    
    aov2 = aov.copy()                                      # Create a copy to avoid modifying the original
    aov2['mean_sq'] = aov2[:]['sum_sq']/aov2[:]['df']      # Calculate mean squares (sum_sq / df) and add as a new column
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)']      # Define the order of columns for the output
    aov2.loc['Total'] = [aov2['sum_sq'].sum(),             # Sum of squares (total sum of sum_sq)
                        aov2['df'].sum(),                  # Degrees of freedom (total sum of df)
                        np.NaN,                            # Mean square is not applicable for the total row
                        np.NaN,                            # F-statistic is not applicable for the total row
                        np.NAN]                            # p-value is not applicable for the total row     
    aov2 = aov2[cols]                                      # Reorder the columns as defined earlier
    
    return aov2

In [13]:
 anova_summary(anova_toasters)

,sum_sq,df,mean_sq,F,PR(>F)
C(toastermodel),886.4,2.0,443.200000,59.357143,1.306568e-10
Residual,201.6,27.0,7.466667,NaN,NaN
Total,1088.0,29.0,NaN,NaN,NaN


#### Using ANOVA to compare the fuel consumption of manual and automatic cars

In [15]:
url1 = ('https://gist.githubusercontent.com/seankross/a412dfbd88b3db70b74b/raw/5f23f993cd87c283ce766e7ac6b329ee7cc2e1d1/mtcars.csv')

cars = pd.read_csv(url1)

cars.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [16]:
# Fit a linear model to predict 'mpg' (miles per gallon) based on the categorical variable 'am' (transmission type)
lm_mpg1 = ols('mpg ~ C(am)', data=cars).fit()  

# Perform a Type II ANOVA on the fitted linear model
anova_mpg1 = sm.stats.anova_lm(lm_mpg1, typ=2)  

# Summarize the ANOVA results using the custom function 'anova_summary'
anova_summary(anova_mpg1)  

,sum_sq,df,mean_sq,F,PR(>F)
C(am),405.150588,1.0,405.150588,16.860279,0.000285
Residual,720.896599,30.0,24.029887,NaN,NaN
Total,1126.047188,31.0,NaN,NaN,NaN


    The p-value (0.000285) is much smaller than 𝛼 (0.05). Thus, we reject the null hypothesis (H0: mu_am = mu_ma). This suggests that the means of mpg differ significantly between the levels of am.

#### Note: F = t^2

In [19]:
# Assuming `anova_mpg1` is the ANOVA table DataFrame
f_value = anova_mpg1.loc['C(am)', 'F']  # Extract the F-statistic for 'C(am)'

# Print the extracted F value
print(f"F-statistic: {f_value:.4f}")

F-statistic: 16.8603


In [20]:
import scipy.stats as sps

automatic = cars[cars['am']==0][['mpg']]
manual = cars[cars['am']==1][['mpg']]

# Perform the two-sample t-test
tstat, p = sps.ttest_ind(automatic, manual, alternative='two-sided', equal_var=True)

# Ensure outputs are scalar
if isinstance(tstat, np.ndarray):
    tstat = tstat.item()
if isinstance(p, np.ndarray):
    p = p.item()

# t^2
t2 = tstat**2

# Print the results
print(f"t-statistic = {tstat:.4f}, p-value = {p:.4f}")
print()
print(f't^2 = {t2:.4f}')

t-statistic = -4.1061, p-value = 0.0003

t^2 = 16.8603


#### We can use ANOVA to see if there are any differences in the mean fuel consumption per number of cylinders.

In [22]:
cars.groupby(['cyl'])['mpg'].mean()

cyl
4    26.663636
6    19.742857
8    15.100000
Name: mpg, dtype: float64

The ANOVA summary for the comparison of the mean fuel consumption per number of cylinders

In [24]:
# Fit a linear model to predict 'mpg' (miles per gallon) based on the categorical variable 'cyl' (number of cylinders)
lm_mpg2 = ols('mpg ~ C(cyl)', data=cars).fit()  

# Perform a Type II ANOVA on the fitted linear model
anova_mpg2 = sm.stats.anova_lm(lm_mpg2, typ=2)  

# Summarize and format the ANOVA results using the custom 'anova_summary' function
anova_summary(anova_mpg2)  

,sum_sq,df,mean_sq,F,PR(>F)
C(cyl),824.784590,2.0,412.392295,39.697515,4.978919e-09
Residual,301.262597,29.0,10.388365,NaN,NaN
Total,1126.047187,31.0,NaN,NaN,NaN


    The p-value (0.0000) << 𝛼 (0.05). Thus, we reject the null hypothesis (H0: all mu(cyl) is equal). This suggests that the means differ significantly between the levels of cylinders.

## Tukey’s Range Test

Tukey's Range Test, also known as **Tukey's Honestly Significant Difference (HSD) Test**, is a post-hoc analysis used after an ANOVA test to determine **which specific group means are significantly different** from each other.

#### When to Use Tukey’s Test:
1. **ANOVA indicates significant differences**:  
   Use Tukey’s test after rejecting the null hypothesis in an ANOVA test.
2. **Multiple group comparisons**:  
   When comparing the means of three or more groups to identify pairwise differences.

---

#### Hypotheses:
- **Null Hypothesis $( H_0 )$**:  
  The means of the two groups being compared are equal.  
 $
  \mu_i = \mu_j
$
- **Alternative Hypothesis $( H_1 )$**:  
  The means of the two groups are not equal.  
  $
  \mu_i \neq \mu_j
$

---

#### How it Works:
1. **Calculate the difference** between the means of all pairs of groups.
2. **Compare the differences** to a critical value (Tukey's HSD) derived from the studentized range distribution:
  $
   HSD = q \cdot \sqrt{\frac{\text{MSE}}{n}}
   $
   Where:
   - $ q $: Studentized range statistic (depends on the number of groups and degrees of freedom).
   - $ \text{MSE} $: Mean Square Error from ANOVA.
   - $ n $: Number of observations per group.

3. If the mean difference exceeds $ HSD $, the pair of means is significantly different.

---

#### Advantages:
- Controls the **family-wise error rate** (probability of making one or more Type I errors in multiple comparisons).
- Suitable for **pairwise comparisons** across multiple groups.

---

#### Limitations:
- Assumes **equal variances** across groups.
- Assumes the **sample sizes are equal**. Unequal sample sizes can reduce accuracy.

---

#### Applications:
- **Experimental studies**: Comparing treatment effects across multiple groups.
- **Agriculture**: Assessing crop yields under different conditions.
- **Marketing**: Evaluating customer preferences for different product variants.


In [28]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Perform Tukey's Honestly Significant Difference (HSD) test
tukey_test = pairwise_tukeyhsd(endog=toasters_melt['excesshours'],     # Dependent variable ('excesshours')
                                groups=toasters_melt['toastermodel'],  # Independent variable ('toastermodel')
                                alpha=0.05)                            # Alpha
print(tukey_test)

  Multiple Comparison of Means - Tukey HSD, FWER=0.05  
  group1  group2 meandiff p-adj  lower    upper  reject
-------------------------------------------------------
centurion  eight     10.4   0.0   7.3701 13.4299   True
centurion   five     -2.0 0.248  -5.0299  1.0299  False
    eight   five    -12.4   0.0 -15.4299 -9.3701   True
-------------------------------------------------------


Significant Differences:

    centurion vs. eight: Significant difference (𝑝 < 0.05).
    eight vs. five: Significant difference (𝑝 < 0.05).

No Significant Difference:

    centurion vs. five: No significant difference (𝑝 > 0.05).

Practical Insight:

    The toaster model eight has significantly higher mean excesshours compared to both centurion and five.

    No meaningful difference is observed between centurion and five.

## Repeated Measures ANOVA (ANOVA RM)

#### What is Repeated Measures ANOVA?
Repeated Measures ANOVA is a statistical test used to compare **means across multiple measurements** taken on the same subjects under different conditions or time points. It accounts for the **correlation between repeated observations** on the same individuals.

---

#### Key Features:
- **Within-Subject Design**: Measures the same individuals multiple times (e.g., at different time points or under varying conditions).
- **Variance Partitioning**: Separates the variance due to **differences between subjects** from the variance due to **differences within subjects**.
- **Tests for Mean Differences**: Identifies whether the mean differences across conditions or time points are statistically significant.

---

#### When to Use:
- To analyze data where the same participants are measured repeatedly, such as:
  - **Longitudinal studies**: Measuring blood pressure at different times.
  - **Experimental studies**: Testing a drug under different doses.
  - **Cognitive studies**: Assessing performance across tasks.

---

#### Hypotheses:
- **Null Hypothesis $( H_0 )$**:  
  There are no mean differences across the repeated measures.  
  $
  \mu_1 = \mu_2 = \mu_3 = \dots
  $
- **Alternative Hypothesis $( H_1 )$**:  
  At least one condition has a different mean.

---

#### Assumptions:
1. **Normality**: The differences between repeated measures are normally distributed.
2. **Sphericity**: The variances of the differences between all pairs of conditions are equal (tested with Mauchly’s test for sphericity).
3. **Independence**: Observations between subjects are independent.

---

#### Advantages:
- Reduces **error variance** caused by individual differences because the same participants are used across conditions.
- Requires fewer participants than a between-subjects ANOVA for similar power.

---

#### Limitations:
- Assumes sphericity; if violated, corrections like **Greenhouse-Geisser** or **Huynh-Feldt** are needed.
- Cannot handle missing data easily.

---

#### Example:
- Measuring the reaction time of participants under three different conditions: **low light**, **medium light**, and **high light**. Repeated Measures ANOVA can determine if the reaction times differ significantly across lighting conditions.


In [31]:
# https://figshare.com/articles/dataset/Starfleet_Headache_Treatment_-_Example_Data_for_Repeated_ANOVA/19089896/1?file=33924695

headache = pd.read_csv('starfleetHeadacheTreatment.csv')

headache.head()

,Volunteer,Kabezine,Headezine,Paramoline,Ibuprenine
0,1,10.0,3.4,1.7,5.7
1,2,7.7,3.9,4.3,4.4
2,3,7.6,7.6,2.5,5.9
3,4,7.1,6.5,9.2,2.9
4,5,10.0,3.4,9.7,3.3


In [32]:
# Reshape the DataFrame from wide format to long format using pandas melt function
h_melt = pd.melt(
    headache.reset_index(),         # Reset the index of the 'headache' DataFrame to include it as a column
    id_vars=['Volunteer'],          # Specify 'Volunteer' as the identifier variable (remains unchanged)
    value_vars=headache.columns,    # Specify the columns to unpivot (all columns in 'headache')
    var_name='drug',                # Name for the new column that stores the original column names
    value_name='health'             # Name for the new column that stores the corresponding values
)

h_melt.head()

,Volunteer,drug,health
0,1,Kabezine,10.0
1,2,Kabezine,7.7
2,3,Kabezine,7.6
3,4,Kabezine,7.1
4,5,Kabezine,10.0


In [33]:
h_melt.tail()

,Volunteer,drug,health
75,16,Ibuprenine,3.8
76,17,Ibuprenine,4.9
77,18,Ibuprenine,4.4
78,19,Ibuprenine,5.3
79,20,Ibuprenine,4.4


In [34]:
from statsmodels.stats.anova import AnovaRM

# Perform a repeated measures ANOVA using statsmodels' AnovaRM function
aovrm = AnovaRM(
    data=h_melt,                  # The long-format DataFrame containing the data
    depvar='health',              # The dependent variable to analyze (column 'health')
    subject='Volunteer',          # The column identifying the subjects (unique for each participant)
    within=['drug']               # The within-subject factor(s) (column 'drug' with repeated measures)
).fit()                           # Fit the model to calculate the ANOVA

print(aovrm)

              Anova
     F Value Num DF  Den DF Pr > F
----------------------------------
drug 19.4528 3.0000 57.0000 0.0000



### Key Results:

#### F Value:
- **$F = 19.4528$**:  
  The F-statistic measures the ratio of variance between the drug groups to the variance within groups.  
  A higher F value indicates greater differences between the groups relative to variability within the groups.

#### Degrees of Freedom:
- **Numerator DF (Num DF)**:  
  $3.0000$: Represents the number of levels in the independent variable minus 1 (in this case, 4 drug types - 1 = 3).
- **Denominator DF (Den DF)**:  
  $57.0000$: Represents the total number of observations minus the number of groups (adjusted for repeated measures).

#### p-value:
- **$Pr > F = 0.0000$**:  
  Indicates that the probability of observing an F value as extreme as $19.4528$, under the null hypothesis (no effect of drug on health), is effectively zero.  
  Since $p < 0.05$, we reject the null hypothesis at a 95% confidence level.

---


#### Conclusion
    The F value (19.4528) and p-value (0.0000) . p-value << alpha , we reject null hypothesis (H0: all means are equal). This indicate that there is a statistically significant difference in health outcomes across the drug groups. At least one drug has a significantly different effect on health.

## Kruskal-Wallis Test – Non-parametric One-Way ANOVA

#### What is the Kruskal-Wallis Test?
The Kruskal-Wallis test is a **non-parametric alternative to One-Way ANOVA**, used to compare the distributions of **three or more independent groups**. Unlike the traditional ANOVA, it does not assume that the data is normally distributed or that the groups have equal variances.

---

#### When to Use:
1. When the **assumptions of One-Way ANOVA** (normality and equal variances) are violated.
2. When the data is **ordinal**, **non-normal**, or contains **outliers**.
3. Comparing **three or more independent groups** (e.g., test scores across multiple classes).

---

#### Hypotheses:
- **Null Hypothesis $( H_0 )$**:  
  All groups have the same distribution.  
  $
  H_0: F_1(x) = F_2(x) = \dots = F_k(x)
  $
  (The population medians of all groups are equal.)
  
- **Alternative Hypothesis $( H_1 )$**:  
  At least one group has a different distribution.

---

#### How it Works:
1. **Ranks the Data**: Combines all groups into a single dataset and assigns ranks to the data points.
2. **Calculates the Test Statistic**:  
   The test statistic, $ H $, is calculated as:
   $
   H = \frac{12}{N(N+1)} \sum_{i=1}^k \frac{R_i^2}{n_i} - 3(N+1)
   $
   Where:
   - $ N $: Total number of observations.
   - $ k $: Number of groups.
   - $ R_i $: Sum of ranks for the $ i $-th group.
   - $ n_i $: Number of observations in the $ i $-th group.

3. **Compare to Chi-Square Distribution**:  
   The test statistic $ H $follows a chi-square distribution with $ k-1 $ degrees of freedom.

---

#### Assumptions:
1. The groups are **independent**.
2. The data is at least **ordinal** (can be ranked).
3. The sample size is **large enough** for the chi-square approximation to be valid.

---

#### Advantages:
- Does not require **normality** or **homogeneity of variance**.
- Works with **ordinal data** and is robust to outliers.

---

#### Limitations:
- Less powerful than ANOVA when data meets parametric assumptions.
- Does not indicate **which groups differ**; post-hoc tests (e.g., Dunn’s Test) are needed for pairwise comparisons.

---

#### Example:
Comparing **customer satisfaction scores** (ranked from 1 to 10) across **three stores** to determine if satisfaction differs significantly between them.

| **Instructor** | **Holographic Communicator** | **Project & Book** |
|:--------------:|:----------------------------:|:------------------:|
| 81.4           | 75.4                        | 85.1               |
| 90.4           | 100.0                       | 95.5               |
| 85.2           | 83.2                        | 93.8               |
| 100.0          | 80.7                        | 100.0              |
| 98.9           | 78.9                        | 88.2               |
| 89.0           | 78.2                        | 91.7               |
| 90.7           | 84.8                        | 93.2               |


In [39]:
instructor = [81.4, 90.4, 85.2, 100.0, 98.9, 89.0, 90.7]
holographic = [75.4, 100.0, 83.2, 80.7, 78.9, 78.2, 84.8]
project_book = [85.1, 95.5, 93.8, 100.0, 88.2, 91.7, 93.2]

df = pd.DataFrame({'instructor': instructor,
                    'holographic': holographic,
                    'project_book': project_book})

df.median()

instructor      90.4
holographic     80.7
project_book    93.2
dtype: float64

In [40]:
from scipy.stats import kruskal

# Kruskal-Wallis test
hstat, p = kruskal(df['instructor'], df['holographic'], df['project_book'])

print('stat = {0:.4f}, p-value= {1:.4f}'.format(hstat, p))

stat = 6.7188, p-value= 0.0348


    With an H statistic of 6.7188 and a p-value lower than 0.05, we reject the null hypothesis for a 95% confidence level and conclude that there are differences in the medians for the three different methods of learning.

## Two-Factor or Two-Way ANOVA

#### What is Two-Way ANOVA?
Two-Way ANOVA (Analysis of Variance) is a statistical method used to examine the effect of **two independent variables (factors)** on a dependent variable. It also analyzes whether there is an **interaction effect** between the two factors.

---

#### When to Use:
1. **Two categorical independent variables** (factors) are influencing a continuous dependent variable.
2. Determine if each factor significantly affects the dependent variable.
3. Assess whether there is an interaction between the two factors that jointly affects the dependent variable.

---

#### Hypotheses:
For **each factor (main effects)**:
- **Null Hypothesis $( H_0 )$**:  
  The means of the dependent variable are the same across all levels of the factor.  
  $
  H_0: \mu_1 = \mu_2 = \dots = \mu_k
  $
- **Alternative Hypothesis $( H_1 )$**:  
  At least one group mean is different.

For the **interaction effect**:
- **Null Hypothesis $( H_0 )$**:  
  There is no interaction between the two factors.
- **Alternative Hypothesis $( H_1 )$**:  
  There is an interaction between the two factors.

---

#### How it Works:
1. **Partitioning Variance**:  
   Two-Way ANOVA partitions the total variance into:
   - Variance due to each factor (main effects).
   - Variance due to the interaction between factors.
   - Residual (error) variance.

2. **F-Test**:  
   For each factor and the interaction, calculate an **F-statistic**:
   $
   F = \frac{\text{Variance due to Factor}}{\text{Error Variance}}
   $
   Compare this to a critical value to determine significance.

---

#### Assumptions:
1. **Independence**: Observations are independent.
2. **Normality**: Dependent variable is approximately normally distributed within each group.
3. **Homogeneity of Variances**: Variance of the dependent variable is equal across all levels of the factors.

---

#### Advantages:
1. Can test the **main effects** of two factors simultaneously.
2. Can identify **interaction effects**, which reveal how the factors jointly influence the dependent variable.
3. Efficient for analyzing data with two categorical variables.

---

#### Limitations:
1. Sensitive to violations of assumptions (e.g., non-normality, unequal variances).
2. Interaction effects can complicate interpretation.

---

#### Example:
**Research Question**:  
A researcher is studying how **study method** (video, textbook) and **study environment** (quiet, noisy) affect test scores. Two-Way ANOVA can:
1. Determine if the **study method** affects scores.
2. Determine if the **study environment** affects scores.
3. Assess whether there is an **interaction** between study method and environment.


In [43]:
# https://figshare.com/articles/dataset/Python_Study_Scores/19208676/1?file=34125654

df = pd.read_csv('Python_Study_Scores.csv')

df.head()

,Delivery,Revision,Score
0,Instructor,Weekly Quiz,81.4
1,Instructor,Mock Exam,90.4
2,Instructor,Weekly Quiz,85.2
3,Instructor,Mock Exam,100.0
4,Instructor,Mock Exam,98.9


In [44]:
# Create a pivot table
table = pd.pivot_table(
    df,                       # Source DataFrame
    values='Score',           # The column to aggregate (dependent variable)
    index=['Delivery'],       # Rows in the pivot table (independent variable 1)
    columns=['Revision'],     # Columns in the pivot table (independent variable 2)
    aggfunc='mean'            # Aggregation function (calculate the mean)
)

# Display the pivot table
table

Revision,Mock Exam,Weekly Quiz
Delivery,,
Holographic,90.85,74.89
Instructor,94.31,86.42
Project,96.28,87.33


In [45]:
# Define the formula for Two-Way ANOVA including main effects and interaction
formula = 'Score ~ C(Delivery) + C(Revision) + C(Delivery):C(Revision)'  
# Dependent variable: 'Score'
# Independent variables: 'Delivery', 'Revision', and their interaction 'Delivery:Revision'

# Fit the model using Ordinary Least Squares (OLS) regression
model = ols(formula, df).fit()

# Perform ANOVA on the fitted model
aov_table = sm.stats.anova_lm(model, typ=2)  # Type II ANOVA calculates sums of squares for each term, adjusting for other terms

# Summarize the ANOVA table with additional metrics
anova_summary(aov_table)

,sum_sq,df,mean_sq,F,PR(>F)
C(Delivery),920.552333,2.0,460.276167,23.759686,3.961052e-08
C(Revision),1793.066667,1.0,1793.066667,92.559000,2.638804e-13
C(Delivery):C(Revision),192.314333,2.0,96.157167,4.963681,1.049756e-02
Residual,1046.096000,54.0,19.372148,NaN,NaN
Total,3952.029333,59.0,NaN,NaN,NaN


#### C(Delivery):

    Sum of Squares (920.55): Represents the variation in scores explained by the Delivery factor.
    Degrees of Freedom (2): Indicates there are 3 groups for the Delivery factor.
    Mean Square (460.28): The average variation attributed to the Delivery factor.
    F-value (23.76): A high F-statistic indicates the Delivery factor significantly affects scores.
    p-value (3.96e-08): The p-value is much smaller than 0.05, meaning the effect of Delivery is statistically significant.
    
#### C(Revision):

    Sum of Squares (1793.07): Represents the variation in scores explained by the Revision factor.
    Degrees of Freedom (1): Indicates there are 2 groups for the Revision factor.
    Mean Square (1793.07): The average variation attributed to the Revision factor.
    F-value (92.56): A very high F-statistic shows the Revision factor has a strong effect on scores.
    p-value (2.64e-13): The effect of Revision is highly statistically significant.
    
#### Interaction (C(Delivery):C(Revision)):

    Sum of Squares (192.31): Represents the variation in scores due to the interaction between Delivery and Revision.
    Degrees of Freedom (2): Indicates there are 3 combinations for the interaction.
    Mean Square (96.16): The average variation attributed to the interaction effect.
    F-value (4.96): Indicates the interaction has a moderate effect.
    p-value (0.0105): The interaction effect is statistically significant (p < 0.05), suggesting the impact of Delivery depends on Revision, or vice versa.
    
#### Residual (Error):

    Sum of Squares (1046.10): The unexplained variation in the data.
    Degrees of Freedom (54): Represents the remaining observations not explained by the factors.
    Mean Square (19.37): The average unexplained variation.
    
#### Total:

    Sum of Squares (3952.03): The total variation in the dataset.
    Degrees of Freedom (59): The total number of observations minus 1.
    
#### Key Takeaways:

    Both Delivery and Revision have significant main effects on the Score (p < 0.05).
    The interaction between Delivery and Revision is also significant (p = 0.0105), indicating the effect of one factor depends on the levels of the other.
    Since the interaction is significant, the main effects should be interpreted with caution as they are not independent of the interaction.

    For a 95% confidence level, since the p-values for Delivery and Revision are both less than 0.05, we can conclude that  both factors have a statistically significant effect on the score, rejecting the null hypothesis that the means for each factor are all equal. We can now turn our attention to the  interaction effect, with F(2, 54) = 4.9636 and p = 0.010, indicating that there is a significant interaction between the delivery and revision methods on the score obtained.

### Add Effect Sizes

In [49]:
# Function to calculate effect sizes (eta-squared and omega-squared) for ANOVA results
def effect_sizes(aov):
    
    aov['eta_sq'] = 'NaN'                                  # Initialize a new column 'eta_sq' with NaN values
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])  # Calculate eta-squared for each row except the last (Residual row)
    mse = aov['sum_sq'].iloc[-1] / aov['df'].iloc[-1]      # Calculate Mean Square Error (MSE) from the residual row
    aov['omega_sq'] = 'NaN'                                # Initialize a new column 'omega_sq' with NaN values
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)  # Calculate omega-squared for each row
    
    return aov                                             # Return the updated ANOVA table with added effect sizes

In [50]:
effect_sizes(aov_table)

,sum_sq,df,F,PR(>F),eta_sq,omega_sq
C(Delivery),920.552333,2.0,23.759686,3.961052e-08,0.232932,0.222040
C(Revision),1793.066667,1.0,92.559000,2.638804e-13,0.453708,0.446617
C(Delivery):C(Revision),192.314333,2.0,4.963681,1.049756e-02,0.048662,0.038669
Residual,1046.096000,54.0,NaN,NaN,NaN,NaN


**Eta-squared** measures the proportion of total variance in the dependent variable that is attributable to a particular factor or interaction in an ANOVA.

**Omega-squared** is an adjusted version of eta-squared that accounts for bias due to sample size, providing a more accurate estimate of the effect size.

#### Example:

If $ \eta^2 = 0.25 $ for a factor, it means 25% of the total variance in the dependent variable is explained by that factor.

If $ \omega^2 = 0.20 $, it suggests a 20% true effect size, accounting for error and sample size adjustments.

## Pearson and Spearman Correlations

In [53]:
import statsmodels.formula.api as smf

source = [30.87, 32.75, 35.75, 35.88, 38.49, 39.87, 40.92, 43.34, 43.88, 43.99, 44.38, 45.31, 45.34, 45.37, 47.08, 
          47.66, 47.74, 48.62, 50.68, 51.11, 52.42, 53.14, 53.50, 54.97, 55.43, 56.48, 57.67, 64.66, 65.23, 65.79]

# Create a DataFrame with independent and dependent variables
data = pd.DataFrame({
    'x': range(1, 31),   # Independent variable (e.g., time or observation number)
    'y': source          # Dependent variable
})

# Fit the Ordinary Least Squares (OLS) regression model
model = smf.ols('y ~ 1 + x', data=data).fit()

# Print the summary of the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     592.6
Date:                Tue, 07 Jan 2025   Prob (F-statistic):           2.21e-20
Time:                        15:30:24   Log-Likelihood:                -61.654
No. Observations:                  30   AIC:                             127.3
Df Residuals:                      28   BIC:                             130.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     32.3799      0.732     44.216      0.0

In [54]:
from scipy.stats import rankdata

data['x_ranked'] = rankdata(data['x'])  # Rank the x values
data['y_ranked'] = rankdata(data['y'])  # Rank the y values

# Perform the OLS regression with ranked data
model = smf.ols('y_ranked ~ 1 + x_ranked', data=data).fit()

print(model.summary())  # Display the results

                            OLS Regression Results                            
Dep. Variable:               y_ranked   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 9.011e+31
Date:                Tue, 07 Jan 2025   Prob (F-statistic):               0.00
Time:                        15:30:24   Log-Likelihood:                 946.38
No. Observations:                  30   AIC:                            -1889.
Df Residuals:                      28   BIC:                            -1886.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   8.438e-15   1.87e-15      4.512      0.0

#### When to Use Ranked Data in OLS

    Robustness: Ranked data minimizes the impact of outliers.
    Non-parametric Regression: If you suspect the data violates assumptions of normality or linearity.
    Ordinal Variables: When x or y represents ranks or ordered categories rather than precise numerical measurements.

In [56]:
# Manual Ranking

def signed_rank(df):
    
    return np.sign(df) * df.abs().rank()

### Manual Signed Rank

- Positive numbers retain their ranks as positive.
- Negative numbers get negative ranks.
- Zeros remain zero.

Step-by-Step Execution:

                 Input Data: [5, -3, 0, 7]
    Compute Absolute Values: [5, 3, 0, 7]
       Rank Absolute Values: [3.0, 2.0, 0.0, 4.0]
       Apply Original Signs: [3.0, -2.0, 0.0, 4.0]

In [58]:
# Example data
data = pd.Series([5, -3, 0, 7])

result = signed_rank(data)

result

0    3.0
1   -2.0
2    0.0
3    4.0
dtype: float64

### Why Use the Wilcoxon Signed-Rank Test?

#### **Purpose**
- To compare two related samples, matched samples, or repeated measurements of the same group.
- It tests whether the **median difference** between paired observations is zero.

---

#### **When to Use**
1. **Non-Normal Data**:  
   When the paired differences between observations are not normally distributed.
2. **Ordinal Data**:  
   Works with data that can be ranked or ordered, even if it isn't numerical.
3. **Small Sample Size**:  
   Effective for small datasets where parametric tests like the paired t-test may not be reliable.

---

#### **Hypotheses**
- **Null Hypothesis $( H_0 )$**:  
  The median difference between the paired samples is zero.  
  $H_0: \text{Median Difference} = 0$
  
- **Alternative Hypothesis $( H_1 )$**:  
  The median difference between the paired samples is not zero.  
  $H_1: \text{Median Difference} \neq 0$

---

#### **Advantages**
- **Non-Parametric**:  
  Does not require the data to follow a normal distribution.
- **Robust**:  
  Can handle outliers better than parametric tests.

---

#### **Applications**
- Comparing **before and after** effects in experiments (e.g., effectiveness of a treatment or tool).
- Evaluating **performance improvements** (e.g., productivity before and after training).
- **Behavioral studies**: Measuring changes in responses over time. 

For example, in a **task completion time study**, we could use the Wilcoxon test to determine if a new tool significantly reduces task completion time compared to before.


**Scenario: A company launches a new productivity tool for its employees and wants to evaluate its impact on the time employees spend completing a standard task. The company measures the time (in minutes) for 10 employees to complete the task before and after introducing the tool. The goal is to determine whether the new tool significantly affects task completion time.**

The company records the following times for the task:

| **Employee** | **Before (Pre-tool)** | **After (Post-tool)** |
|--------------|------------------------|------------------------|
| 1            | 25                     | 22                     |
| 2            | 30                     | 28                     |
| 3            | 20                     | 22                     |
| 4            | 27                     | 25                     |
| 5            | 24                     | 22                     |
| 6            | 29                     | 30                     |
| 7            | 35                     | 32                     |
| 8            | 40                     | 38                     |
| 9            | 50                     | 45                     |
| 10           | 22                     | 20                     |


In [62]:
from scipy.stats import wilcoxon

# Data
before = [25, 30, 20, 27, 24, 29, 35, 40, 50, 22]
after = [22, 28, 22, 25, 22, 30, 32, 38, 45, 20]

# Calculate differences
data = pd.DataFrame({'Before': before, 'After': after})
data['Difference'] = data['Before'] - data['After']

# Perform Wilcoxon Signed-Rank Test
stat, p = wilcoxon(data['Before'], data['After'])
print(f"Wilcoxon statistic: {stat:.4f}, p-value: {p:.4f}\n")

# Interpretation
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: The new tool significantly affects task completion time.")
else:
    print("Fail to reject the null hypothesis: The new tool does not significantly affect task completion time.")

Wilcoxon statistic: 5.5000, p-value: 0.0273

Reject the null hypothesis: The new tool significantly affects task completion time.
